General Overview:

1. Parse program `argv n` times to list of `(prog, i) : BLOCK * Int`.
    - `BLOCK = list[SMTM]`
    - `i` is number of unravellings of loops (need to consider cases 0 to `argv n`)
    - Convert to static single assignment (i.e each var instance is unique)
2. Create list of symbolic execution outcomes of each `(prog, n)` as `(P, C) : z3.formula * z3.formula`
    - Each if statement creates a new element in the list (i.e. function will be recursive, each if statement creates two branches of the function concatenated together)
3. Solve each `(P, C)` by passing `!P && C` through z3.
    - if `SAT`, need to find example output that does not work. z3 already provides this. 



In [ ]:
import typing

# stmt := x ":=" AEXP ";"
# | x, y := AEXP "," AEXP ";"
# | x "[" AEXP "]" ":=" AEXP ";"
# | "if" BEXP "then" BLOCK "else" BLOCK "end"
# | "if" BEXP "then" BLOCK "end"
# | "while" BEXP "do" BLOCK "end"
# | "assert" ASSN ";"
block = list[stmt]



def see() :
    parsedProg : list[block, int] = parse(program)
    noViolations = True
    for (prog, n) in parsedProg :
        for symbOut in symbolic_outcomes(prog, z3.true(), z3.true()) : # formula with just true
            s = Tactic('smt').solver()
            s.add(symbOut)
            if s.check() == sat :
                noViolations = False
                print_true_vars(s.model()) # double check
                break # needs to break out of inner for-loop
    
    if noViolations :
        print("No Violations Found")

            

In [9]:
VARS : dict = {}

# for parser
def addvar(var : str) -> str :
    if var in VARS :
        VARS[var] += 1
        return var + str(VARS[var])
    else :
        VARS[var] = 0
        return var + '0'






In [47]:
def symbolic_outcomes(blk : block, accl, accr : z3.formula) -> list[z3.formula] :
    if len(blk) == 0 :
        return [Implies(accl, accr)]
    else :
        hd = blk[0]
        if (isinstance(hd, IF)) : # placeholder class syntax for now
            symbolic_outcomes(blk[-1], And(accl, hd.ifcond), And(accr, hd.then)) + \
                symbolic_outcomes(blk[-1], And(accl, Not(hd.ifcond)), And(accr, hd.els))
        elif (isinstance(hd, ASSRT)) :
            symbolic_outcomes(blk[-1], accl, And(accr, Not(hd.cond))) # not sure about this
        elif (isinstance(hd, ASSGN)) :
            symbolic_outcomes(blk[-1], accl, And(accr, hd.cond)) # cond is just assignment here

2

In [ ]:
def print_true_vars(m : z3.model) :
    acc = []
    for d in m.decls():
        if d.name[-1] == "0" && d.name[-2] #is not number :
            acc += m[d]
    print(PROGRAM_NAME, acc)

In [78]:
from typing import *
from typeguard import check_type

STMT = Union[int, str]

x : STMT = 2

UserId = NewType('IF', int)

print(type(UserId))

try:
    check_type('if', x, int)
    print("string_list conforms to string_list_class")
except TypeError:
    print("string_list does not conform to string_list_class")

<class 'function'>
string_list conforms to string_list_class


In [70]:
!pip install typeguard

Defaulting to user installation because normal site-packages is not writeable


In [25]:
from z3 import z3core
z3

<module 'z3' from '/home/sam/.local/lib/python3.9/site-packages/z3/__init__.py'>

In [28]:
from z3 import *
 
def main(): 
    
    s = Solver()
    x = Int('x')
    y = Int('y')
    s.add(x < 10)

main()

NameError: name 'Solver' is not defined

In [44]:
a = []
len(a)

0

In [95]:
from z3 import Int


ImportError: cannot import name 'Int' from 'z3' (/home/sam/.local/lib/python3.9/site-packages/z3/__init__.py)